In [14]:
import hydra
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from cpprb import ReplayBuffer
from jaxman.env.env import JaxMANEnv
from jaxman.planner.rl_planner.memory.cpprb_utils import (
    _sample_experience as cpp_sample,
)
from jaxman.planner.rl_planner.memory.dataset import Buffer
from jaxman.planner.rl_planner.memory.utils import _sample_experience as original_sample
from jaxman.utils import compute_agent_action
from omegaconf import OmegaConf


config = hydra.utils.instantiate(OmegaConf.load("../scripts/config/env/continuous.yaml"))
env = JaxMANEnv(config)

In [15]:
# initialized original buffer
original_buffer = Buffer(env.observation_space, env.action_space, 1000000)
original_buffer.size=10000

In [17]:
# initialized cpp buffer
obs_dim = env.observation_space["obs"].shape[0]
num_agents = env.observation_space["comm"].shape[0]
comm_dim = env.observation_space["comm"].shape[1]
mask_dim = env.observation_space["mask"].shape[0]
total_obs_dim = obs_dim + num_agents * comm_dim + mask_dim
act_dim = 2

env_dict = {
            "obs": {"shape": total_obs_dim},
            "act": {"shape": act_dim},
            "reward": {},
            "next_obs": {"shape": total_obs_dim},
            "mask": {},
        }
cpp_buffer = ReplayBuffer(1000000, env_dict)

obs = np.zeros((10000, total_obs_dim))
act = np.zeros((10000,2))
reward = np.zeros((10000,))
mask = np.zeros((10000,))
cpp_buffer.add(obs=obs, act=act, next_obs=obs, reward=reward, mask=mask)

0

In [19]:
%timeit original_sample(original_buffer, 1000, num_agents, comm_dim)

142 µs ± 2.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
%timeit cpp_sample(cpp_buffer, False, False, 1000, num_agents, comm_dim)

296 µs ± 3.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
